<a href="https://colab.research.google.com/github/Kianjputnam/project_chd/blob/main/Final_Paper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Coronary Heart Disease Project
## by Eleanor Byrd, Ashley Whitehouse, Kian Putnam

 ### **Summary**

 The focus of this project was to build predictive algorithms that predict how likely it is for someone to develop coronary heart disease

### **Data**

The data, which was collected through the Framingham Heart Study, has several variables that are believed to be related to the development of cardiovascular disease. The cleaning of this data can be found in Chd_Cleaning.ipynb. The variables we decided to use in our predictive model are listed below, along with how they were cleaned for our analysis. For categorical variables, we decided to drop all 'na' values since they can be problematic in predictive models. This only removed 3.3% of the testing data and 3.8% of the training data. We also decided to map the variable encodings to their coresponding categorical values in new columns, so that our models can run with the one-hot encodings and the graphs can be made with categorical values.

For numerical variables, we decided to all 'na' values into the median value for the variable.

sex:

This variable is the sex of the patient, where 0 indicates female and 1 indicates male. This variable had no 'na' values to drop. In a new variable called 'sex_category', all of the 0 values were mapped to 'female' and all of the 1 values were mapped to 'male'.

age:

This variable is the age, in years, of the patient's medical evaluation. There were no 'na' values to drop, so no changes were made to the age data.

education:

This variable is the patient's highest level of education that has been completed. The 'na' values were dropped. In a new variable called 'edu_category', all of the numerical values were mapped to the propper categories. All of the 1 values were mapped to 'some HS', the 2 values were mapped to 'HS/GED, the 3 values were mapped to 'some college/vocation', and the 4 values were mapped to 'college'.

currentSmoker:

This variable indicates wheter the patient currently smokes, where 0 means no and 1 means yes. There were no 'na' values to drop. In a new variable called 'smoker_category', all 0 values were mapped to 'no smoking' and all 1 values were mapped to 'smoking'.

cigsPerDay:

This variable is the number of cigarettes the patient smokes per day. There were 5 'na' values for this variable. Each missing value was replaced with the median number of cigarettes smoked per day.

BPMeds:

This variable indicates whether a patient used anti-hypertensive medication at the time they were examined. The 'na' values were dropped. In a new variable 'BPMeds_category', 0 values were mapped to 'no meds' and 1 values were mapped to 'on meds'.

prevalentStroke:

This indicates whether the patient has had a stroke. There were no 'na' values to drop. In a new variable 'stroke_category', 0 values were mapped to 'no stroke' and 1 values were mapped to 'had stroke'.

prevalentHyp:

This indicates whether the patient was hypertensive. There were no 'na' values to drop. In a new variable 'hyp_category', 0 was mapped to 'no hyp' and 1 was mapped to 'hyp'.

diabetes:

This shows whether the patient was diabetic. There were no 'na' values to drop. In a new variable called 'diabetes_category', 0 values were mapped to 'no diabetes' and 1 values were mapped to 'diabetes'.

totChol:

This variable is the patient's cholesterol (mg/dl) at the time of the examination. There were 11 'na' values for this variable. Each missing value was replaced with the median total cholesterol.

sysBP:

This is the patient's systolic blood pressure (mmHg) at the time of the examination. There were no 'na' values to drop and no mapping was needed for this numerical variable.

diaBP:

This is the patient's diastolic blood pressure (mmHg) at the time of the examination. There were no 'na' values to drop and no mapping was needed for this numerical variable.

BMI:

This is the patient's body mass index (weight (kg)/height (m)^2) at the time of the examination. There were 4 'na' values for this variable. Each missing value was replaced with the median BMI.

heartRate:

This is the patient's heart rate (beats/minute) at the time of the examination. There was only 1 'na' value for this variable and it was replaced with the median heart rate.

glucose:

This is the patient's glucose level (mg/dl) at the time of the examination. There were 101 'na' values for this variable. Each missing value was replaced with the median glucose value.

TenYearCHD:

This indicates whether the patient had coronary heart disease in 10 years. There were no 'na' values to drop. In a new variable called 'chd_category', the 0 values were mapped to 'no chd' and the 1 values were mapped to 'chd'.

